In [12]:
import os
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [23]:
# 데이터 전처리
data_dir = './data'
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 색상, 밝기, 대비 변화
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
   transforms.Resize((224, 224)),
   transforms.ToTensor(),
   transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [24]:
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [26]:
class EyeClassifier(nn.Module):
    def __init__(self, pretrained_model, num_classes):
        super(EyeClassifier, self).__init__()
        self.pretrained_model = pretrained_model
        self.mlp = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.pretrained_model(x)
        x = x.view(x.size(0), -1)
        x = self.mlp(x)
        return x


pretrained_model = torchvision.models.resnet50(pretrained=True)
num_classes = len(train_dataset.classes)
model = EyeClassifier(pretrained_model, num_classes)

c:\Users\daeho\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\daeho\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [27]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [28]:
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total
    print(f'Epoch {epoch+1} Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')


Epoch 1 Loss: 2.8351, Train Accuracy: 5.90%
Epoch 2 Loss: 2.7841, Train Accuracy: 5.18%
Epoch 3 Loss: 2.7797, Train Accuracy: 5.39%
Epoch 4 Loss: 2.7777, Train Accuracy: 6.67%
Epoch 5 Loss: 2.7779, Train Accuracy: 5.80%
Epoch 6 Loss: 2.7747, Train Accuracy: 6.31%
Epoch 7 Loss: 2.7760, Train Accuracy: 5.60%
Epoch 8 Loss: 2.7748, Train Accuracy: 6.83%
Epoch 9 Loss: 2.7758, Train Accuracy: 7.65%
Epoch 10 Loss: 2.7751, Train Accuracy: 6.62%


In [29]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
   for inputs, labels in test_loader:
       inputs, labels = inputs.to(device), labels.to(device)
       outputs = model(inputs)
       _, predicted = torch.max(outputs.data, 1)
       total += labels.size(0)
       correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 8.13%
